<a href="https://colab.research.google.com/github/ARGF0RCE/Shiny-TF-DC-UC-Resources/blob/main/tensorflow_nlp_fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP Fundamentals in TensorFlow

NLP has the goal of deriving information out of natural language

Another common term for NLP problems is sequence to sequence or seq2sqeq problems.

## Check for GPU

In [1]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-f76b6d83-6dc5-a226-ce58-cbd17e93c64d)


## Get helper functions

In [2]:
!wget -O helper_functions.py https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import series of helper functions for the notebook.
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2022-07-28 17:35:27--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-07-28 17:35:27 (71.1 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get a text dataset

The dataset we're going to be using is Kaggle's introduction to NLP dataset

See the original source [here](https://www.kaggle.com/c/nlp-getting-started)

In [3]:
!wget -O nlp_getting_started.zip https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# Unzip data
unzip_data("nlp_getting_started.zip")

--2022-07-28 17:35:30--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.196.128, 74.125.31.128, 74.125.134.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.196.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2022-07-28 17:35:30 (140 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Become one with the data

### Visualizing a text dataset

To visualize outr text samples, we first have to read them in.

We are going to use `pandas`.

In [4]:
import pandas as pd
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
train_df["text"][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [6]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [7]:
# What does the test dataframe look like
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [8]:
# How many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [9]:
# How many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [10]:
# Let's visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-5) # create random indexing
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
    _, text, target = row
    print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
    print(f"Text:\n{text}\n")
    print("---\n")

Target: 0 (not real disaster)
Text:
@ego_resolution im glad. My gay can heal 1000 wounds

---

Target: 1 (real disaster)
Text:
Humboldt Cty Sheriff's Office has issued an evacuation advisory for 10 residence in the Lassics area... more info at http://t.co/ERUzBUQZYU

---

Target: 1 (real disaster)
Text:
y'all read 12000 Nigerian refugees repatriated from Cameroon http://t.co/aVwE1LBvhn

---

Target: 1 (real disaster)
Text:
@Bill_Roose That looks so desolate and just...depressing

---

Target: 1 (real disaster)
Text:
Blood Memory: Intragenerational Trauma and the Death of Sandra Bland http://t.co/ZWeyGpHpf7

---



## Split data into training and validation splits

In [11]:
from sklearn.model_selection import train_test_split
# Use train_test _split to split trauiing data into training and validation datasets.
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled['text'].to_numpy(),
                                                                           train_df_shuffled['target'].to_numpy(),
                                                                           test_size=0.1, # Use 10 percent of training data
                                                                           random_state=42)

In [12]:
# Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [13]:
# Check the first ten samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

## Convertinfg text into numbers

When dealing with a text problem, one of the first things you'll have to do before you can build a model is to conver your text to numbers.

There are a few ways to do this, namely:
* Tokenization
* Embedding

### Text vectorization (tokenization)

In [14]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [15]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use the default TextVectorization parameters
text_vectorizer =   TextVectorization(max_tokens=None, # how many words in the vocabulary (automatically add <OOV>)
                                      standardize="lower_and_strip_punctuation",
                                      split="whitespace",
                                      ngrams=None, # Create groups of n-words
                                      output_mode="int",
                                      output_sequence_length=None,
                                      pad_to_max_tokens=False)

In [16]:
# Find the average number of tokens (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [17]:
# Setup text verctorization variables
max_vocab_length = 10000
max_length = 15

text_vectorizer = TextVectorization(max_tokens=max_vocab_length, # how many words in the vocabulary (automatically add <OOV>)
                                      standardize="lower_and_strip_punctuation",
                                      split="whitespace",
                                      ngrams=None, # Create groups of n-words
                                      output_mode="int",
                                      output_sequence_length=max_length,
                                      pad_to_max_tokens=True)

In [18]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [19]:
# Create a sample a sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [20]:
# Choos e a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"""
Original text:\n{random_sentence}\n
Vectorized version of the same: {text_vectorizer([random_sentence])}
""")


Original text:
New Explosion-proof Tempered Glass Screen Protector Film for Blackberry Z10 - Full read byÛ_ http://t.co/tOYU16mxBO http://t.co/P10hNDc0Mm

Vectorized version of the same: [[  50 2466 2263 1847 1481 2336  953   10 2540 2212  120  193 2171    1
     1]]



In [21]:
# Get the unique words in the vacbulary
words_in_vocab = text_vectorizer.get_vocabulary() # Get all of the unique words
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f"""
Number of words in vocab: {len(words_in_vocab)}
5 most common words: {top_5_words}
5 least common words: {bottom_5_words}""")


Number of words in vocab: 10000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating an Embedding using an Embedding layer from TensorFlow

[Source of infromation](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding)

The parameters we care most about our embedding layer:
* `input_dim` - The size of the vocabulary
* `output_dim` - the size of the output embedding vector.
* `input_length` - length of sequences being passed to the embedding layer

In [22]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             input_length=max_length)

embedding

In [23]:
# Get a random sentence from the training sentence
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
        \n\nEmbedded version:")
# Embed the random sentence (turn it into dense vectors of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
Your brain is particularly vulnerable to trauma at two distinct ages http://t.co/KnBv2YtNWc @qz @TaraSwart @vivian_giang        

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.03159674,  0.042063  ,  0.03860866, ...,  0.00515585,
         -0.0478061 , -0.02163512],
        [ 0.02166207,  0.01381565,  0.02941534, ...,  0.03762202,
          0.01133548,  0.0252416 ],
        [ 0.00930892, -0.03391455, -0.03815402, ...,  0.02377484,
         -0.03981777,  0.03063435],
        ...,
        [-0.02148486, -0.01549617,  0.00810302, ..., -0.04635366,
         -0.0180442 ,  0.01647725],
        [-0.02891125,  0.03869127,  0.00543829, ..., -0.02734613,
         -0.03734737,  0.00668765],
        [ 0.00678771,  0.01087432, -0.03816639, ..., -0.01075663,
         -0.01859407,  0.03008112]]], dtype=float32)>

In [24]:
# Check out a single token's embedding
sample_embed[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([-0.03159674,  0.042063  ,  0.03860866, -0.00799295, -0.00561793,
        0.04370028, -0.00895634, -0.04024635,  0.00478507,  0.04508055,
        0.02255804,  0.03967838,  0.01056993,  0.00151964, -0.01770402,
       -0.00378197, -0.03618523,  0.00582341,  0.0394768 ,  0.00517253,
       -0.0216183 ,  0.00140767, -0.04952307, -0.01957102, -0.00935075,
        0.01057345,  0.01866659,  0.00179399,  0.03921219, -0.04541754,
        0.01635149, -0.02973733,  0.03738343, -0.00136159, -0.02338998,
        0.03474832,  0.03826172, -0.01194106, -0.03512741, -0.02976062,
        0.01754415,  0.04737738,  0.01088022,  0.00046735,  0.00188833,
       -0.00736697,  0.02467411,  0.0167856 ,  0.03745664, -0.01908265,
        0.02959125, -0.01465613, -0.04708693, -0.04144418,  0.02663196,
       -0.01469241, -0.0456411 , -0.02787384,  0.04837639, -0.0416906 ,
        0.02376262, -0.03007858,  0.04565508, -0.01724205,  0.04027087,
        0.038273

## Modelling experiments we're running

| Experiment Number | Model                                       |
|-------------------|---------------------------------------------|
| 0                 | Naive Bayes with TF-IDF encoder (baseline)  |
| 1                 | Feed-forward neural network (dense model)   |
| 2                 | LSTM (RNN)
| 3                 | GRU (RNN)                                   |
| 4                 | Bidirectional-LSTM (RNN)                    |
| 5                 | 1D Convolutional Neural Network             |
| 6                 | TensorFlow Hub Pretrained Feature Extractor |
| 7                 | TensorFlow Hub Pretrained Feature Extractor (10% of the data) |

### Model 0: Getting a baseline model

As with all machine learning modelling experiments, it's important to create a baseline model so you've got a benchmark for future experiments to build upon.

For the baseline, we'll be using scikit-learn's Mutinomial Naive Bayes Machine Learning Problem.

> 🔑 **Note**: It's common practice non-DL algorithms baseline because of their speed and then later using DL to see if you can improve upon.

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
# Create tokenization and modelling pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
                    ("clf", MultinomialNB()) # model the text
])
# Fit the pipline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [26]:
# Evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model an accuracy of: {baseline_score*100:.2f}%")

Our baseline model an accuracy of: 79.27%


In [27]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [28]:
from sklearn import metrics
def evaluation_metrics(y_pred, y_true):
    accuracy = metrics.accuracy_score(y_true, y_pred)
    precision = metrics.precision_score(y_pred=y_pred, y_true=y_true)
    recall = metrics.recall_score(y_true=y_true, y_pred=y_pred)
    f1_score = metrics.f1_score(y_true, y_pred)
    results = {
    'accuracy': accuracy,
    'precision': precision,
    'recall': recall,
    'F1-Score': f1_score
    }
    return results


### Creating an evaluation function for our modelling experiments

In [29]:
baseline_results = evaluation_metrics(y_pred=baseline_preds, y_true=val_labels)
baseline_results

{'F1-Score': 0.734006734006734,
 'accuracy': 0.7926509186351706,
 'precision': 0.8861788617886179,
 'recall': 0.6264367816091954}

### Model 1: Feed forward neural network

In [30]:
# Create a tensorboard callback
from helper_functions import create_tensorboard_callback
# Create a directory to save tensorflow logs
SAVE_DIR = "model_logs"

In [31]:
# import tensorflow as tf
# tf.random.set_seed(42)
# # Using Sequential API
# model_1 = tf.keras.Sequential([
#                                tf.keras.layers.Input(shape=(1,), dtype=tf.string),
#                                text_vectorizer,
#                                tf.keras.layers.Embedding(input_dim=max_vocab_length, output_dim=128, input_length=max_length),
#                                tf.keras.layers.Dense(128, activation='relu'),
#                                tf.keras.layers.Dense(128, activation='relu'),
#                                tf.keras.layers.Dense(128, activation='relu'),
#                                tf.keras.layers.Dense(1, activation='sigmoid')
# ])

In [32]:
# model_1.compile(loss=tf.keras.losses.BinaryCrossentropy(),
#                 optimizer=tf.keras.optimizers.Adam(),
#                 metrics=['accuracy'])

In [33]:
# model_1_history = model_1.fit(train_sentences, tf.expand_dims(train_labels, axis=-1), epochs=10, validation_data=(val_sentences, tf.expand_dims(val_labels, axis=-1)), validation_steps=len(val_sentences))

In [34]:
# Using the functional API
from tensorflow.keras import layers
tf.random.set_seed(42)
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_1 = tf.keras.Model(inputs, outputs, name="model_1")

In [35]:
model_1.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 dense (Dense)               (None, 15, 128)           16512     
                                                                 
 dense_1 (Dense)             (None, 15, 128)           16512     
                                                                 
 dense_2 (Dense)             (None, 15, 128)           16512     
                                                           

In [36]:
model_1.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [37]:
model_1_history = model_1.fit(train_sentences, tf.expand_dims(train_labels, axis=-1), epochs=10, validation_data=(val_sentences, tf.expand_dims(val_labels, axis=-1)), validation_steps=len(val_sentences),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model_1")])

Saving TensorBoard log files to: model_logs/model_1/20220728-173536
Epoch 1/10
215/215 [==============================] - 8s 16ms/step - loss: 0.5239 - accuracy: 0.7313 - val_loss: 0.4555 - val_accuracy: 0.7913
Epoch 2/10
215/215 [==============================] - 3s 12ms/step - loss: 0.3175 - accuracy: 0.8701 - val_loss: 0.4992 - val_accuracy: 0.7756
Epoch 3/10
215/215 [==============================] - 3s 12ms/step - loss: 0.2110 - accuracy: 0.9180 - val_loss: 0.5594 - val_accuracy: 0.7756
Epoch 4/10
215/215 [==============================] - 3s 12ms/step - loss: 0.1464 - accuracy: 0.9432 - val_loss: 0.6812 - val_accuracy: 0.7743
Epoch 5/10
215/215 [==============================] - 3s 12ms/step - loss: 0.1087 - accuracy: 0.9568 - val_loss: 0.7971 - val_accuracy: 0.7677
Epoch 6/10
215/215 [==============================] - 3s 12ms/step - loss: 0.0937 - accuracy: 0.9644 - val_loss: 0.8676 - val_accuracy: 0.7598
Epoch 7/10
215/215 [==============================] - 3s 12ms/step - loss:

In [38]:
# Evaluate the model on the validation dataset
model_1.evaluate(val_sentences, tf.expand_dims(val_labels, axis=-1))

24/24 [==============================] - 0s 3ms/step - loss: 1.1834 - accuracy: 0.7612


[1.1833590269088745, 0.76115483045578]

In [39]:
# Make some predictions
model_1_pred_probs = model_1.predict(tf.expand_dims(val_sentences, axis=-1))
model_1_pred_probs.shape

(762, 1)

In [40]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [41]:
model_1_results = evaluation_metrics(y_true=val_labels, y_pred=model_1_preds)
model_1_results

{'F1-Score': 0.7083333333333333,
 'accuracy': 0.7611548556430446,
 'precision': 0.8007246376811594,
 'recall': 0.6350574712643678}

In [42]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False,  True, False])

### Visualizing learned Embeddings

In [43]:
# Get the vocabulary from text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab)

10000

In [44]:
# Model 1 summary
model_1.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 dense (Dense)               (None, 15, 128)           16512     
                                                                 
 dense_1 (Dense)             (None, 15, 128)           16512     
                                                                 
 dense_2 (Dense)             (None, 15, 128)           16512     
                                                           

Now we've got the embedding matrix our model has learned to represent our tokens, let's see how we can visulaize it.

To do so, TensorFlow has a handy tool called projector: http://projector.tensorflow.org/

And TensorFlow also has an incredible guide on word embeddings themselves: https://www.tensorflow.org/text/guide/word_embeddings

In [46]:
# Get the weight matrix of embedding layer
# (these are the numerical representations of each token in our training data, which have been trainied for 10 epochs)
embed_weights = model_1.get_layer("embedding").get_weights()[0]
embed_weights

array([[ 0.02076538,  0.00742582, -0.05012066, ..., -0.00311943,
        -0.05290118, -0.02843167],
       [ 0.01203756, -0.0574736 ,  0.03790959, ...,  0.05463247,
         0.01412099, -0.09128877],
       [-0.00865737, -0.0313861 , -0.06268649, ..., -0.00248528,
         0.02706851, -0.03673403],
       ...,
       [ 0.0300361 ,  0.03913939, -0.02133331, ..., -0.0272535 ,
        -0.0090941 , -0.04984508],
       [-0.0333124 ,  0.04667868,  0.01286821, ..., -0.01406243,
        -0.04576761, -0.028202  ],
       [-0.08650982,  0.12072483,  0.11125153, ...,  0.014571  ,
        -0.006509  ,  0.10475975]], dtype=float32)

In [47]:
print(embed_weights.shape)

(10000, 128)


In [48]:
# Create embedding files
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [49]:
# Download files from colab to upload to projector
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Recurrent Neural Network (RNN)

RNN's are useful for sequence data.

The premise of a recurrent neural network is to use the representation of a previous input to aid the reporesentation of a later input.

